In [1]:
import os, time, random, logging , cv2 , csv
import numpy as np
#import mtcnn

#import pandas as pd
import tensorflow as tf
print("tf",tf.version.VERSION)
#os.system("cat /usr/local/cuda/version.txt")
#os.system("nvcc --version\n")
os.system("conda list | grep -E 'tensorflow|cudnn|cudatoolkit|numpy'")

from tensorflow import keras
#from keras import backend as K

#from keras import models, layers, backend as K
#from keras.layers import Activation

#from tensorflow.keras.callbacks import ModelCheckpoint
#from tensorflow.keras.utils import get_custom_objects
#from keras.utils.generic_utils import get_custom_objects
from tqdm.keras import TqdmCallback


from utils import tf_formh5 , xdv , globo , auxua as aux



''' GPU CONFIGURATION '''

tf_formh5.set_tf_loglevel(logging.ERROR)
tf_formh5.tf.debugging.set_log_device_placement(False) #Enabling device placement logging causes any Tensor allocations or operations to be printed.
tf_formh5.set_memory_growth()
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

tf 2.2.0
cudatoolkit               10.1.243            h8cb64d8_11    conda-forge
cudnn                     7.6.5.32             hc0a50b0_1    conda-forge
neptune-tensorflow-keras  2.1.0              pyhd8ed1ab_0    conda-forge
numpy                     1.19.5           py38hc896f84_4  
numpy-base                1.19.5           py38h21a3de8_4  
tensorflow                2.2.0           gpu_py38hb782248_0    anaconda
tensorflow-base           2.2.0           gpu_py38h83e3d50_0    anaconda
tensorflow-estimator      2.6.0            py38h709712a_0    conda-forge
tensorflow-gpu            2.2.0                h0d30ee6_0    anaconda


/home/jtstudents/anaconda3/envs/zugpu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cudatoolkit               10.1.243            h8cb64d8_11    conda-forge
cudnn                     7.6.5.32             hc0a50b0_1    conda-forge
neptune-tensorflow-keras  2.1.0              pyhd8ed1ab_0    conda-forge
numpy                     1.19.5           py38hc896f84_4  
numpy-base                1.19.5           py38h21a3de8_4  
tensorflow                2.2.0           gpu_py38hb782248_0    anaconda
tensorflow-base           2.2.0           gpu_py38h83e3d50_0    anaconda
tensorflow-estimator      2.6.0            py38h709712a_0    conda-forge
tensorflow-gpu            2.2.0                h0d30ee6_0    anaconda

Num GPUs Available:  4
4 Physical GPUs, 4 Logical GPUs


In [2]:
''' TRAIN CONFIG '''

in_height = 120; in_width = 160
train_rgb_config = {
    "ativa" : 'relu',
    "optima" : 'sgd',
    "batch_type" : 0,   # =0 all batch have frame_max or video length // =1 last batch has frame_max frames // =2 last batch has no repetead frames
    "frame_max" : '1000',
    "ckpt_start" : f"{0:0>8}",  #used in train_model: if 00000000 start from scratch, else start from ckpt with config stated
    "epochs" : 30
}

train_wav_config = {
    "nada" : 0
}

In [3]:
"""" TRAIN FILES """

train_fn, train_labels, valdt_fn, valdt_labels = xdv.train_valdt_files()

update_index_train = range(0, len(train_fn))
update_index_valdt = range(0, len(valdt_fn))


full_train_fn (3953,) 
full_train_normal_fn (2048,) 
full_train_abnormal (1905,)

train_fn (3162,) 
train_normal_fn (1613,) 
train_abnormal_fn (1549,)

valdt_fn (791,) 
valdt_normal_fn (435,) 
valdt_abnormal_fn (356,)


In [ ]:
""" INPUT DATA"""

def input_train_video_data(file_name):
    print("\n\ninput_train_video_data\n")
    
    video = cv2.VideoCapture(file_name)
    total_frame = video.get(cv2.CAP_PROP_FRAME_COUNT)
    
    divid_no = 1
    
    frame_max = train_rgb_config["frame_max"]
    
    # define the nmbers of batchs to divid atual video (divid_no)
    if total_frame > int(frame_max):
        total_frame_int = int(total_frame)
        if total_frame_int % int(frame_max) == 0:
            divid_no = int(total_frame / int(frame_max))
        else:
            divid_no = int(total_frame / int(frame_max)) + 1
     
        
    # if normal puts the video at the random start frame
    if 'Normal' in file_name:
        print("\n\nNORMAL\n\n")
        if divid_no != 1:
            slice_no = int(random.random()*divid_no)
            passby = 0
            if slice_no != divid_no - 1:
                while video.isOpened and passby < int(frame_max) * slice_no:
                    passby += 1
                    success, image = video.read()
                    if success == False:
                        break
            else:
                while video.isOpened and passby < total_frame - int(frame_max):
                    passby += 1
                    success, image = video.read()
                    if success == False:
                        break
     
                    
    batch_no = 0
    batch_frames = []
    batch_frames_flip = []
    counter = 0
    while video.isOpened:               
        success, image = video.read() #pixels in image are uint8 [0,255]
        if success == False:
            break
        
        image = cv2.resize(image, (in_width, in_height))
        image_flip = cv2.flip(image, 1)
        
        image_array = np.array(image)/255.0
        image_array_flip = np.array(image_flip)/255.0
        
        batch_frames.append(image_array)
        batch_frames_flip.append(image_array_flip)
        
        counter += 1
        if counter == int(frame_max):
            break
            
    video.release()
    batch_frames = np.array(batch_frames)

    print("\n\tdata",file_name,"\n\ttotal_frames=",total_frame,"\n\tbatch_frames.shape=",np.shape(np.expand_dims(batch_frames,0)),"\n")    
    return np.expand_dims(batch_frames,0), np.expand_dims(batch_frames_flip, 0), total_frame

def generate_input(data,update_index,validation):
    #has_visited = [0 for i in range(len(train_fn))]
    data_var_name = [k for k, v in globals().items() if v is data][0]
    print("\n\nGENERATE_INPUT FOR",data_var_name,\
        '\n\tupdate_index len = ',len(update_index),\
        '\n\tdata len = ',len(data))
    
    loop_no = 0
    while 1:
        index = update_index[loop_no]
        loop_no += 1
        print("\n",data_var_name," index",index," loop_no",loop_no)
        if loop_no == len(data):loop_no= 0
        
        
        batch_frames, batch_frames_flip, total_frames = input_train_video_data(data[index])
        print("\n\t",data_var_name,"data[",index,"]=",data[index],"\n\ttotal_frames=",total_frames,"\n\tbatch_frames.shape=",batch_frames.shape,"\n")
        
        
        if not validation:
            #batch_frames
            if 'label_A' in data[index]: yield batch_frames, np.array([0])   #normal
            else: yield batch_frames, np.array([1])   #abnormal
            
            #batch_frames_flip
            if 'label_A' in data[index]: yield batch_frames_flip, np.array([0])  #normal
            else: yield batch_frames_flip, np.array([1])  #abnormal
        else:
            #batch_frames
            if 'label_A' in data[index]: yield batch_frames, np.array([0])   #normal
            else: yield batch_frames, np.array([1])   #abnormal
                
    print("\nloop_no=",loop_no)


In [ ]:
batch_frames, batch_frames_flip, total_frames = input_train_video_data(train_fn[333])

In [4]:
'''' MODEL '''

def all_operations(args):
    x = args[0]#;tf.print(x.shape)
    x = tf.reshape(x, [1, -1,x.shape[1]*x.shape[2]*x.shape[3]])
    return x
@tf.function
def loss_category(y_true, y_pred):    
    #tf.print(y_pred, y_true, 'Prediction')
    cce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    return cce

def gelu(x): return 0.5*x*(1+tf.tanh(np.sqrt(2/np.pi)*(x+0.044715*tf.pow(x, 3))))
#get_custom_objects().update({'gelu': Activation(gelu)})


def form_model(params):
    print("\nFORM_MODEL\n")
    
    if params["ativa"]=='leakyrelu': ativa = keras.layers.LeakyReLU()
    elif params["ativa"]=='gelu': ativa = gelu
    elif params["ativa"]=='relu': ativa = 'relu'
    else: raise Exception("no ativa named assim")
    
    
    image_input = keras.Input(shape=(None, in_height, in_width, 3))
    # ( 1 , frame_max , h , w , ch)    
    # ( 1 , 1000 , 120 , 160 , 3)
    c3d_mp = keras.Sequential([
        keras.layers.Conv3D(4,(2,3,3), activation=ativa),   #c3d_layer1
        keras.layers.MaxPooling3D((1,2,2)),                 #c3d_pooling1

        keras.layers.Conv3D(8,(4,3,3), activation=ativa),   #c3d_layer2
        keras.layers.MaxPooling3D((2,2,2)),                 #c3d_pooling2

        keras.layers.Conv3D(16,(8,3,3), activation=ativa),  #c3d_layer3
        keras.layers.MaxPooling3D((4,2,2))                  #c3d_pooling3
    ])
    c3d_mp_out = c3d_mp(image_input)
    # ( 1 , time_steps , spatl_featr1 , spatl_featr2 , spatl_featr3 ) 
    # ( 1 , 122 , 13 , 18 , 16 )  
    
    c3d_mp_flatten = keras.layers.Lambda(all_operations)(c3d_mp_out)  # flatten spatial features to time series
    # ( 1 , time_steps , spatl_featr_flattned ) 
    # ( 1 , 122        , 3744 )
    
    lstm1 = keras.layers.LSTM(1024, return_sequences=True)(c3d_mp_flatten) #input_shape=(120,c3d_mp_flatten.shape[2]),
    # ( 1 , time_steps , units) 
    # ( 1 , 122        , 1024 ) 
    
    global_rgb_feature = keras.layers.GlobalMaxPooling1D()(lstm1)
    # ( 1 , 1024 ) 


    ''' waves coming '''
    sinet_aas_len = 8
    aas_input = tf.keras.layers.Input(shape=(None, sinet_aas_len)) #(TIMESTEPS,AAS)
    gloabl_aas = keras.layers.GlobalMaxPooling1D()(aas_input) 
    
    
    av_fusion = keras.layers.concatenate([global_rgb_feature, gloabl_aas], axis=1)


    dense_1 = keras.layers.Dense(128, activation=ativa)(av_fusion)
    soft_max = keras.layers.Dense(1, activation='sigmoid')(dense_1)    
    
    model = keras.Model(inputs=[image_input , aas_input], 
                        outputs=[soft_max])
    model.summary()
   
   
    if params["optima"]=='sgd':optima = keras.optimizers.SGD(learning_rate = 0.0002)
    elif params["optima"]=='adam':optima = keras.optimizers.Adam(learning_rate = 0.0002)
    elif params["optima"]=='adamamsgrad':optima = keras.optimizers.Adam(learning_rate = 0.0002,amsgrad=True)
    else: raise Exception("no optima named assim")

    METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

    model.compile(optimizer=optima, 
                    loss= 'binary_crossentropy', 
                    #loss_weights = class_weights,
                    #metrics=['accuracy']
                    metrics=METRICS)
    
    print("\n\t",params,"\n\n\tOPTIMA",optima,"\n\tATIVA",ativa)
    
    return model


model = form_model(train_rgb_config)


FORM_MODEL

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 120, 1 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, None, 13, 18, 10612       input_1[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (1, None, 3744)      0           sequential[0][0]                 
__________________________________________________________________________________________________
lstm (LSTM)                     (1, None, 1024)      19533824    lambda[0][0]                     
_________________________________________________________________________________

In [ ]:
output = model.predict(batch_frames)
print(np.shape(output))

In [ ]:
#https://www.perplexity.ai/search/035a3df9-b2c0-45bc-908f-aaebd9773117?s=c
# writes only the frist conv3d layer ritgh to video
'''
LAYER = 1
LAYER_NAME = 'c3d_pooling1'

frames = np.concatenate(output, axis=0)

# create video writer object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # specify codec
fps = 30  # specify frame rate
video_writer = cv2.VideoWriter(str(LAYER)+'_'+str(LAYER_NAME)+'_output.mp4', fourcc, fps, (158, 118))

# write frames to video file
for frame in frames:
    frame = np.uint8(frame * 255)  # convert to 8-bit unsigned integer
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # convert from RGB to BGR
    video_writer.write(frame)

# release video writer object
video_writer.release()'''